In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# TODO: Change this to your Drive folder location
WORKING_DIRECTORY = '/content/drive/MyDrive/spacy_intent'

os.chdir(WORKING_DIRECTORY)


In [ ]:
!ls

customer_data.csv  reddit_data.csv


In [ ]:
import pandas as pd
pd.options.display.max_colwidth = None
pd.options.display.max_rows = 6
data = pd.read_csv("customer_data.csv")
data

,text,category,intent
0,I have problems with canceling an order,ORDER,cancel_order
1,how can I find information about canceling orders?,ORDER,cancel_order
2,I need help with canceling the last order,ORDER,cancel_order
...,...,...,...
8172,how can I check the status of the refund?,REFUND,track_refund
8173,help checking if there is anything wrong with my refund,REFUND,track_refund
8174,anything new on my refund?,REFUND,track_refund


In [ ]:
# Let's also store the subreddits which are going to be our labels for the text classification component
cats = data.intent.unique().tolist()
cats

['cancel_order',
 'change_order',
 'change_shipping_address',
 'check_cancellation_fee',
 'check_invoice',
 'check_payment_methods',
 'check_refund_policy',
 'complaint',
 'contact_customer_service',
 'contact_human_agent',
 'create_account',
 'delete_account',
 'delivery_options',
 'delivery_period',
 'edit_account',
 'get_invoice',
 'get_refund',
 'newsletter_subscription',
 'payment_issue',
 'place_order',
 'recover_password',
 'registration_problems',
 'review',
 'set_up_shipping_address',
 'switch_account',
 'track_order',
 'track_refund']

In [ ]:
!python -m spacy download en_core_web_md

2024-01-19 06:53:46.568896: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-19 06:53:46.568943: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-19 06:53:46.570408: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-19 06:53:46.578242: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-19 06:53:47.727314: W tensorflow/compiler/tf2

In [ ]:
from typing import Set, List, Tuple
from spacy.tokens import DocBin
import spacy

nlp = spacy.load("en_core_web_md")

def make_docs(data: List[Tuple[str, str]], target_file: str, cats: Set[str]):
    docs = DocBin()
    for doc, label in nlp.pipe(data, as_tuples=True):

        for cat in cats:
            doc.cats[cat] = 1 if cat == label else 0
        docs.add(doc)
    docs.to_disk(target_file)
    return docs

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(data["text"].values, data["intent"].values, test_size=0.2)
make_docs(list(zip(X_train, y_train)), "train.spacy", cats=cats)
make_docs(list(zip(X_valid, y_valid)), "valid.spacy", cats=cats)

In [ ]:
!pip install spacy-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.9/197.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 9.3 MB/s eta 0:00:00


In [ ]:
#! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

2024-01-19 06:57:24.880449: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-19 06:57:24.880512: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-19 06:57:24.882054: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-19 06:57:26.418120: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
! python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy --gpu-id 0

2024-01-19 06:59:24.604276: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-19 06:59:24.604320: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-19 06:59:24.605551: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-19 06:59:25.685352: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initializ

In [ ]:
import spacy

In [ ]:
nlp_ner1 = spacy.load("model-best")


In [ ]:

!ls

base_config.cfg  customer_data.csv  model-best	reddit_data.csv  valid.spacy
config.cfg	 model		    model-last	train.spacy


In [ ]:
doc=nlp_ner1('''where is my order?''')

In [ ]:
output=doc.cats

print(output)

{'cancel_order': 5.538594223253313e-07, 'change_order': 4.656888279441773e-07, 'change_shipping_address': 2.070063516157461e-07, 'check_cancellation_fee': 4.563295874504547e-08, 'check_invoice': 1.3012680710744462e-07, 'check_payment_methods': 8.94749447866161e-08, 'check_refund_policy': 2.796353548717434e-08, 'complaint': 8.625541170204087e-08, 'contact_customer_service': 1.1395263044278181e-07, 'contact_human_agent': 1.3618665661851992e-07, 'create_account': 1.5562233102173195e-07, 'delete_account': 2.871904136725334e-08, 'delivery_options': 5.7282093735011586e-08, 'delivery_period': 1.3159318257294217e-07, 'edit_account': 1.7186030731863866e-07, 'get_invoice': 1.0726717647457917e-07, 'get_refund': 6.030011689972525e-08, 'newsletter_subscription': 2.1661870164280117e-07, 'payment_issue': 3.2426882512481825e-08, 'place_order': 8.430295395100984e-08, 'recover_password': 1.8939624624181306e-07, 'registration_problems': 5.499673960684959e-08, 'review': 1.0589096888224958e-07, 'set_up_shi

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U 'spacy[cuda12x]'
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.
2024-01-19 06:12:34.364368: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-19 06:12:34.364436: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attemptin